#### 가위, 바위, 보 데이터를 읽을 수 있는 함수 

In [74]:
def load_data(img_path):
    number_of_data = 300   # 폴더 안에 있는 가위바위보 이미지 개수 총합
    img_size = 28   # 이미지 크기
    color = 3   # 흑백 이미지 = 0, 칼라 이미지 = 3
    
    # 이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs = np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels = np.zeros(number_of_data,dtype=np.int32)
    
    # 확장자 불러오기
    temp = glob.glob(img_path + "/scissor/*.jpeg")
    if len(temp) == 0:
        temp = glob.glob(img_path+"/scissor/*.jpg")   
        if len(temp) == 0:
            extension = "png"
        else:
            extension = "jpg"
    else:
        extension = "jpeg"

    idx = 0
    for file in glob.iglob(img_path+'/scissor/*.{}'.format(extension)):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 0   # 가위 : 0
        idx = idx + 1

    for file in glob.iglob(img_path+'/rock/*.{}'.format(extension)):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 1   # 바위 : 1
        idx = idx + 1       
    
    for file in glob.iglob(img_path+'/paper/*.{}'.format(extension)):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 2   # 보 : 2
        idx = idx + 1

    print("이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/exploration/rock_scissor_paper/trainData"
(x_train, y_train) = load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))


이미지 개수는 300 입니다.
x_train shape: (300, 28, 28, 3)
y_train shape: (300,)


In [75]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

print('Model에 추가된 Layer 개수: ', len(model.layers))

model.summary()

Model에 추가된 Layer 개수:  7
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 100)               80100     
_________________________________________________________________
dense_15 (Dense)             (

In [76]:
import matplotlib.pyplot as plt

x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 3)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

results = model.fit(x_train_reshaped, y_train, epochs=20)


Epoch 1/20
10/10 [==============================] - 0s 2ms/step - loss: 1.1273 - accuracy: 0.3433
Epoch 2/20
10/10 [==============================] - 0s 1ms/step - loss: 1.1084 - accuracy: 0.3367
Epoch 3/20
10/10 [==============================] - 0s 1ms/step - loss: 1.0772 - accuracy: 0.3967
Epoch 4/20
10/10 [==============================] - 0s 1ms/step - loss: 1.0483 - accuracy: 0.6933
Epoch 5/20
10/10 [==============================] - 0s 1ms/step - loss: 1.0065 - accuracy: 0.6433
Epoch 6/20
10/10 [==============================] - 0s 1ms/step - loss: 0.9406 - accuracy: 0.7133
Epoch 7/20
10/10 [==============================] - 0s 2ms/step - loss: 0.8264 - accuracy: 0.7433
Epoch 8/20
10/10 [==============================] - 0s 1ms/step - loss: 0.6863 - accuracy: 0.7467
Epoch 9/20
10/10 [==============================] - 0s 1ms/step - loss: 0.5879 - accuracy: 0.7633
Epoch 10/20
10/10 [==============================] - 0s 1ms/step - loss: 0.4780 - accuracy: 0.7833
Epoch 11/20
10/10 [

In [77]:
# x_test, y_test를 만들기
image_dir_path = os.getenv("HOME") + "/aiffel/exploration/rock_scissor_paper/testData"
(x_test, y_test) = load_data(image_dir_path)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

x_test_reshaped = x_test_norm.reshape( -1, 28, 28, 3)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.

# 모델 시험
test_loss, test_accuracy = model.evaluate(x_test_reshaped, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

이미지 개수는 300 입니다.
10/10 - 0s - loss: 2.8366 - accuracy: 0.2367
test_loss: 2.8365931510925293 
test_accuracy: 0.23666666448116302


# 성능이 나빠서 데이터의 양을 추가

## 1) 데이터 늘리기

In [78]:
# 폴더 내에 있는 모든 이미지 불러오기
image_dir_path = os.getenv("HOME") + "/aiffel/exploration/rock_scissor_paper/data_6600"
dir_list = os.listdir(image_dir_path)

first = True
for dir in dir_list:
    if first == True:
        (x_data, y_data) = load_data(image_dir_path + "/{}".format(dir))
        first = False
    else:
        (x_data_temp, y_data_temp) = load_data(image_dir_path + "/{}".format(dir))
        x_data = np.vstack((x_data, x_data_temp))     
        y_data = np.hstack((y_data, y_data_temp))

print("x_data shape: {}".format(x_data.shape))
print("y_data shape: {}".format(y_data.shape))

이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
이미지 개수는 100 입니다.
이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
이미지 개수는 200 입니다.
이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
이미지 개수는 300 입니다.
x_data shape: (6600, 28, 28, 3)
y_data shape: (6600,)


## 2) 데이터 나누고 정규화

In [79]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.1, shuffle = True, random_state = 42)
x_train_norm = x_train / 255.0
x_test_norm = x_test / 255.0

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(5940, 28, 28, 3)
(5940,)
(660, 28, 28, 3)
(660,)


## 3) 모델 학습

In [80]:
x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 3)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

results = model.fit(x_train_reshaped, y_train, epochs=20)

'''
# epoch에 따른 훈련 데이터 정확성 그래프
plt.plot(results.history['accuracy'])
#plt.plot(results.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()  
'''

Epoch 1/20
186/186 [==============================] - 0s 1ms/step - loss: 1.0433 - accuracy: 0.5162
Epoch 2/20
186/186 [==============================] - 0s 1ms/step - loss: 0.6913 - accuracy: 0.7047
Epoch 3/20
186/186 [==============================] - 0s 1ms/step - loss: 0.4771 - accuracy: 0.8165
Epoch 4/20
186/186 [==============================] - 0s 1ms/step - loss: 0.3499 - accuracy: 0.8699
Epoch 5/20
186/186 [==============================] - 0s 1ms/step - loss: 0.2534 - accuracy: 0.9072
Epoch 6/20
186/186 [==============================] - 0s 1ms/step - loss: 0.1950 - accuracy: 0.9291
Epoch 7/20
186/186 [==============================] - 0s 1ms/step - loss: 0.1720 - accuracy: 0.9382
Epoch 8/20
186/186 [==============================] - 0s 1ms/step - loss: 0.1176 - accuracy: 0.9611
Epoch 9/20
186/186 [==============================] - 0s 1ms/step - loss: 0.0996 - accuracy: 0.9697
Epoch 10/20
186/186 [==============================] - 0s 1ms/step - loss: 0.0838 - accuracy: 0.9732

"\n# epoch에 따른 훈련 데이터 정확성 그래프\nplt.plot(results.history['accuracy'])\n#plt.plot(results.history['val_acc'])\nplt.title('model accuracy')\nplt.ylabel('accuracy')\nplt.xlabel('epoch')\nplt.legend(['train'], loc='upper left')\nplt.show()\n"

## 4) 모델 평가

In [81]:
x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 3)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.

# 모델 시험
test_loss, test_accuracy = model.evaluate(x_test_reshaped, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

21/21 - 0s - loss: 0.0796 - accuracy: 0.9818
test_loss: 0.07961215078830719 
test_accuracy: 0.9818181991577148


## [느낀점]
정확도가 제자리걸음이라 퍼실님께 도움을 요청했는데 데이터의 양을 추가하면 정확도가 높아진다고 힌트를 주셔서 양을 늘렸더니 성능이 향상되었다.

데이터의 양이 학습에 중요한것을 배웠다.